# A model for spinal cord outgrowth and growth fraction

In [1]:
import scipy as sp
import scipy.integrate
import scipy.optimize
import scipy.stats
import gc


import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import random
import multiprocessing
pool = multiprocessing.Pool()
%load_ext ipycache
import probfit
import collections
import functools
import iminuit

def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

/home/fabian/anaconda2/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/fabian/anaconda2/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


## Model definition

In [2]:
def lg_model(model_input, time):
    """ integrates the model ODEs
        
        model_input is expected to be a dict containting the entries
        L0, G0, Tct, k and v. 
        L0  ... float, initial regeneration zone length
        G0  ... float, intitial growth fraction
        Tct ... function of t, returning a float
        k   ... float, activation rate
        v   ... float, clone displacement speed at L0
    
        time ... array of timepoints for which to solve the ODE. First
                 entry is expected to be the time when Nc0 was measured.
    """
    L0 = model_input['L0']
    G0 = model_input['G0']
    Tct = model_input['Tct']
    k = model_input['k']
    v = model_input['v']
    
    def ode_rhs((Lout, G), t0):
        dLout_dt = sp.log(2) / Tct(t0) * (Lout + L0) * G + v
        dG_dt = (1 - G) * (k + sp.log(2) / Tct(t0) * G)
        return [dLout_dt, dG_dt]
    
    LG = sp.integrate.odeint(ode_rhs, (0.0, G0), time)
    
    L = LG[:,0]
    G = LG[:,1]
    
    return L, G

In [3]:
def modelint(model_input, time):
    """ integrates the model ODEs and returns the solution as linear
        interpolation functions.
        
        model_input is expected to be a dict containting the entries
        L0, G0, Tct, k and v. 
        L0  ... float, initial regeneration zone length
        G0  ... float, intitial growth fraction
        Tct ... function of t, returning a float
        k   ... float, activation rate
        v   ... float, clone displacement speed at L0
    
        time ... array of timepoints for which to solve the ODE. First
                 entry is expected to be the time when Nc0 was measured.
    """
    Lp, Gp = lg_model(model_input, time)
    
    L = lambda t: sp.interp(t, time, Lp)
    G = lambda t: sp.interp(t, time, Gp)
    
    return pd.Series({'L': L, 'G': G})

In [4]:
def plot_solution(time, solution):
    fig, ax = plt.subplots(ncols = 2, sharex = True)
    ax[0].plot(time, solution['L'](time))
    ax[0].set_ylabel('L')
    ax[1].plot(time, solution['G'](time))
    ax[1].set_xlabel('time')
    ax[1].set_ylabel('G')
    ax[1].set_ylim(-0.05, 1.05)
    
    return fig, ax

## Data

Source zone length

In [5]:
L0 = pd.read_csv('../../data/source_zone_length.csv').iloc[0,0]

outgrowth

In [6]:
outgrowth = pd.read_csv('../../data/outgrowth.csv')
outgrowth_pop = outgrowth.groupby('time').agg(['mean', 'sem'])['length'].reset_index()

Cell number data

In [7]:
cell_number_data = pd.read_csv('../../data/cell_number_data.csv')

BrdU data (bootstrap result)

In [8]:
TC6dlist = pd.read_csv('../../data/TC6d.csv', squeeze = True)

Velocity

In [9]:
clone_velocities = pd.read_csv('clone_velocities.csv')

Class that handels the data:

In [10]:
class Data:
    """ This class handles the raw data and produces partial model inputs
        from these.
    """
    def __init__(self, raw_data):
        L0 = raw_data['L0']
        cell_number_data = raw_data['cell_number_data']
        cell_number_data = cell_number_data.query('pos >= - @L0').copy()
        cell_number_data['G'] = cell_number_data['PCNA'] / cell_number_data['SOX2']
        cell_number_data['mi'] = cell_number_data['m'] / cell_number_data['PCNA']

        IDtime = {}
        sametimeIDs = {}
        for i, ID in enumerate(cell_number_data['ID'].unique()):
            IDtime[ID] = float(cell_number_data.query('ID == @ID')['time'].unique())
            sametimeIDs[ID] = list(cell_number_data[['time', 'ID']].drop_duplicates().query('time == @IDtime[@ID]')['ID'])
        

        data = raw_data.copy()
        data['cell_number_data'] = cell_number_data
        data['sametimeIDs'] = sametimeIDs
        data['IDtime'] = IDtime
        data['G_data'] = cell_number_data[['ID', 'G']].dropna().reset_index(drop=True).set_index('ID')
        data['G_data']['time'] = [IDtime[ID] for ID in data['G_data'].index]
        data['mi_data'] = cell_number_data[['ID', 'mi']].dropna().reset_index(drop=True).set_index('ID')
        data['mi_data']['time'] = [IDtime[ID] for ID in data['mi_data'].index]
        
        data['G_data_mean'] = data['G_data'].groupby(level = 'ID').agg('mean').groupby('time').agg(['mean', 'std'])
        data['mi_data_mean'] = data['mi_data'].groupby(level = 'ID').agg('mean').groupby('time').agg(['mean', 'std'])
        
        # displacement velocities (= influx at L0)
        clone_velocities = raw_data['clone_velocities']
        data['influx'] = clone_velocities.query('pos == -@L0')['v'].copy()
    
        
        self.data = data
        
    def sample_v(self):
        
        v = self.data['influx'].sample(frac=1.0, replace = True).mean()
        return v
    
    def resample_raw_data(self):
        """ generate case resampled data from the raw data for bootstrapping
        """
        sampled_data = {}
        
        # Source IDs for resampling
        sourceIDs = {}
        sametimeIDs = self.data['sametimeIDs']
        for i, (ID, sametimeID) in enumerate(sametimeIDs.iteritems()):
            sourceIDs[ID] = random.choice(sametimeID)

        
        # L0
        sampled_data['L0'] = self.data['L0']
        
        # mi and G
        sampled_data['mi_data'] = self.create_sampled_data(self.data['mi_data'], 'mi', sourceIDs, )
        sampled_data['G_data'] = self.create_sampled_data(self.data['G_data'], 'G', sourceIDs)
        
        # v
        sampled_data['v'] = self.sample_v()
        
        # Tc6d
        sampled_data['Tc6d'] = self.data['Tc6d'].sample().iloc[0]
        
        return sampled_data
        
    def create_sampled_data(self, data, obs, sourceIDs):
        sampled_data = []
        IDtime = self.data['IDtime']
        for i, sourceID in enumerate(sourceIDs.itervalues()):
            for j in range(len(data.loc[sourceID])):
                line = [i, IDtime[sourceID], data.loc[sourceID][obs].sample()[sourceID]]
                sampled_data.append(line)
        return pd.DataFrame(sampled_data, columns = ['ID', 'time', obs]).sort_values(by = ['time', 'ID']).set_index('ID')
    
    def para_and_G(self, sampled_data, basal_div):
        para_and_G = {}
        
        # regeneration zone
        L0 = sampled_data['L0']
        para_and_G['L0'] = L0
        
        
        # growth fraction
        G_data = sampled_data['G_data']
        
        
        # intitial growth fraction
        initial_cnd = G_data.query('time == 0').copy()
        G0 = float(initial_cnd['G'].groupby(level = 'ID').mean().mean())
        para_and_G['G0'] = G0
        
        # growth fraction timecourse
        later_cnd = G_data.query('0 < time < 8').copy()
        para_and_G['Gexp'] = later_cnd.groupby(level = 'ID').mean().groupby('time').mean()
         
        
        # displacement velocity
        para_and_G['v'] = sampled_data['v']
        
        # cell cycle length timecourse
        Tc6d = sampled_data['Tc6d']
        
        mi_data = sampled_data['mi_data']
        
        Tc_data = mi_data.query('time <= 6').copy()
        Tc_data = Tc_data.groupby(level = 'ID').mean().groupby('time').mean()
        mi6d = Tc_data.loc[6, 'mi']
        Tc_data['Tc'] = Tc6d * mi6d / Tc_data['mi'] 
        Tc_data = Tc_data['Tc'].reset_index()
        
        if basal_div:
            time = [0.0]
            Tc = [Tc_data.query('time <= 3')['Tc'].mean()]
            
            Tct = lambda t, tp = time, Tcp = Tc: sp.interp(t, tp, Tcp)
        else:
            switchtime = sp.random.uniform(3.0, 4.0)
            time = [switchtime,switchtime + 1e-20]
            Tc = [Tc_data.query('time <= 3')['Tc'].mean(), Tc_data.query('time > 3')['Tc'].mean()]

            Tct = lambda t, tp = time, Tcp = Tc: sp.interp(t, tp, Tcp)
        
        para_and_G['Tct'] =  Tct
       
        return para_and_G
    
    
    def fit_activation(self, para_and_G):
        """ fits the activation rate and returns the full model input.
        """

        def f(time, k):
            model_input = para_and_G.copy()
            model_input['k'] = k
            return modelint(model_input, sp.linspace(0, 8, 100))['G'](time)

        xdata = sp.array(para_and_G['Gexp'].reset_index()['time'])
        ydata = sp.array(para_and_G['Gexp']['G'])
        
        def SSE(k):
            return sp.sum((f(xdata, k) - ydata)**2)
        k = sp.optimize.brute(SSE, (slice(0, 1, 0.1), ))[0]
        
        model_input = para_and_G
        model_input['k'] = k

        return model_input
    
    def create_model_input(self, basal_div):
        sampled_data = self.resample_raw_data()
        para_and_G = self.para_and_G(sampled_data, basal_div)
        model_input = self.fit_activation(para_and_G)
        return model_input
        
    
    def sample_inputs(self, N, basal_div = False):
        model_inputs = [self.create_model_input(basal_div) for i in range(N)]
        return pd.DataFrame(model_inputs)

## Model ensemble

In [11]:
def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

class Model_ensemble:
    """ can handle a list of model inputs, solves the model for all of them
        and can describe the ensemble output (e.g. ensemble mean, std, ...)
    """

    def __init__(self, model_inputs):
        time = sp.linspace(0, 8, 100)
        self.model_outputs = pd.concat( (pd.DataFrame(model_inputs),\
                                         pd.DataFrame(model_inputs).apply(modelint, axis =1, args = (time, )) ), axis = 1)
    
    @memoize    
    def describe(self, time,fig, ax):
        ax[0].set_xlim(0, 9)
        for i, obs in enumerate(['L', 'G']):
            sol = self.describe_function_series(time, self.model_outputs[obs])
            ax[i].set_xlabel('time')
            ax[i].plot(time, sol['mean'], 'b')
#             ax[i].plot(time, sol['16.0'], 'b--')
#             ax[i].plot(time, sol['84.0'], 'b--')
#             ax[i].plot(time, sol['2.5'], 'b-.')
#             ax[i].plot(time, sol['97.5'], 'b-.')
            ax[i].plot(time, sol['0.1'], 'b:')
            ax[i].plot(time, sol['99.8'], 'b:')
#             ax[i].plot(time, sol['max'], 'k-.')
#             ax[i].plot(time, sol['min'], 'k-.')
            ax[i].set_ylabel(obs)
        

    def describe_initials(self):
        print 'L0: {0}'.format(self.model_outputs['L0'].iloc[0])
        ax = {}
        ax['G0'] = plt.subplot2grid((2, 3), (0, 0))
        ax['k']  = plt.subplot2grid((2, 3), (0, 1))
        ax['v']  = plt.subplot2grid((2, 3), (0, 2))
        ax['Tc'] = plt.subplot2grid((2, 3), (1, 0), colspan = 3)
        
        for parameter in ['G0', 'k', 'v']:
            self.model_outputs[parameter].hist(ax = ax[parameter])
            ax[parameter].set_xlabel(parameter)
        
        time = sp.linspace(0, 8, 100)
        Tc_desc = self.describe_function_series(time, self.model_outputs['Tct'])
        ax['Tc'].plot(time, Tc_desc['mean'], 'b')
        ax['Tc'].plot(time, Tc_desc['2.5'], 'b--')
        ax['Tc'].plot(time, Tc_desc['97.5'], 'b--')
#         ax['Tc'].plot(time, Tc_desc['mean'] + Tc_desc['std'], 'b--')
#         ax['Tc'].plot(time, Tc_desc['mean'] - Tc_desc['std'], 'b--')
        ax['Tc'].set_xlabel('time')
        ax['Tc'].set_ylabel('Tc')
        plt.gcf().set_size_inches(12, 6)
    
    @memoize
    def evaluate_function_series(self, time, function_series):
        df = function_series.apply(lambda f: pd.Series(f(time))).transpose()
        df.index = time
        df.index.name = 'time'
        return df
    
    @memoize  
    def describe_function_series(self, time, function_series):
        eval_ = self.evaluate_function_series(time, function_series)
        desc = pd.DataFrame(eval_.mean(axis=1), columns = ['mean'])
        desc['std'] = eval_.std(axis = 1)
        desc['sem'] = eval_.sem(axis = 1)
        desc['max'] = eval_.max(axis = 1)
        desc['min'] = eval_.min(axis = 1)
        for alpha in [68.0, 95.0, 99.7]:
            q1 = 0.5 * (100.0 - alpha)
            q2 = 100.0 - q1
            for q in [q1, q2]:
                desc['{0:.1f}'.format(q)] = eval_.apply(lambda x: sp.percentile(x, q), axis = 1)
        return desc
    

In [12]:
def best_fit(model_ensemble, outgrowth_data, cell_number_data):
    mod_out = model_ensemble.model_outputs
    outgrowth_data = outgrowth_data.query('mean != 0')
    
    cell_number_data['G'] = cell_number_data['PCNA'] / cell_number_data['SOX2']
    Gdata = cell_number_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem'])
    Gdata.columns = Gdata.columns.droplevel()
    Gdata = Gdata.reset_index()
    Gdata = Gdata.query('time < 8')

    def chi2L(L, outgrowth_data):
        return probfit.Chi2Regression(L, sp.array(outgrowth_data['time']), sp.array(outgrowth_data['mean']), error=sp.array(outgrowth_data['sem']))()
    
    def chi2G(model_output):
        return probfit.Chi2Regression(model_output['G'],\
                                   sp.array(Gdata['time']),\
                                   sp.array(Gdata['mean']),\
                                   sp.array(Gdata['sem']))()
        

    chi2Ls = mod_out['L'].apply(chi2L, args = (outgrowth_data, ))
    chi2Gs = mod_out.apply(chi2G, axis = 1)
    chi2s = chi2Ls + chi2Gs
    bf = mod_out.loc[chi2s.idxmin()]
    return bf

### Full model

In [ ]:
raw_data = {'L0': L0,
            'cell_number_data': cell_number_data,
            'Tc6d': TC6dlist,
            'clone_velocities': clone_velocities}

data = Data(raw_data)

In [ ]:
%%cache model_inputs.pkl model_inputs --cachedir ../cache/lg_model/
%%time
model_inputs = data.sample_inputs(100000)

In [ ]:
%%cache model_ensemble.pkl model_ensemble --cachedir ../cache/lg_model/
%%time
model_ensemble = Model_ensemble(model_inputs)

In [ ]:
model_ensemble.describe_initials()

In [ ]:
bfcase = best_fit(model_ensemble, outgrowth_pop, cell_number_data)


time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
fig.patch.set_alpha(1.0)
model_ensemble.describe(time, fig, ax)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'],
               label = 'exp (fab)', fmt = 's', color = 'black')

# plot bestfits
ax[0].plot(time, bfcase['L'](time), color = 'green', label = 'bestfit')
ax[1].plot(time, bfcase['G'](time), color = 'green')

ax[0].set_xlim(-0.2, 8.2)
fig.set_size_inches(5, 4)
fig.patch.set_alpha(1.0)
plt.tight_layout()
plt.show()

In [ ]:
bfcase

In [ ]:
gc.collect()

## Basal level of divisions

In [ ]:
%%cache model_inputs_basal_div.pkl model_inputs_basal_div --cachedir ../cache/lg_model/
%%time
model_inputs_basal_div = data.sample_inputs(100000, basal_div = True)

In [ ]:
%%cache model_ensemble_basal_div.pkl model_ensemble_basal_div --cachedir ../cache/lg_model/
%%time
model_ensemble_basal_div = Model_ensemble(model_inputs_basal_div)

In [ ]:
model_ensemble_basal_div.describe_initials()

In [ ]:
bfcase = best_fit(model_ensemble_basal_div, outgrowth_pop, cell_number_data)


time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
fig.patch.set_alpha(1.0)
model_ensemble_basal_div.describe(time, fig, ax)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'],
               label = 'exp (fab)', fmt = 's', color = 'black')

# plot bestfits
ax[0].plot(time, bfcase['L'](time), color = 'green', label = 'bestfit')
ax[1].plot(time, bfcase['G'](time), color = 'green')

ax[0].set_xlim(-0.2, 8.2)
fig.set_size_inches(5, 4)
fig.patch.set_alpha(1.0)
plt.tight_layout()
plt.show()

In [ ]:
gc.collect()

## No divisions

In [ ]:
raw_data_nodiv = {'L0': L0,
            'cell_number_data': cell_number_data,
            'Tc6d': pd.Series([sp.inf]),
            'clone_velocities': clone_velocities}

data_nodiv = Data(raw_data_nodiv)

In [ ]:
%%cache model_inputs_nodiv.pkl model_inputs_nodiv --cachedir ../cache/lg_model/
%%time
model_inputs_nodiv = data_nodiv.sample_inputs(100000)

In [ ]:
%%cache model_ensemble_nodiv.pkl model_ensemble_nodiv --cachedir ../cache/lg_model/
%%time
model_ensemble_nodiv = Model_ensemble(model_inputs_nodiv)

In [ ]:
model_ensemble_nodiv.describe_initials()

In [ ]:
bfcase = best_fit(model_ensemble_nodiv, outgrowth_pop, cell_number_data)

In [ ]:
time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
model_ensemble_nodiv.describe(time, fig, ax)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'], label = 'exp (fab)',
               fmt = 's', color = 'black')

# plot bestfits
ax[0].plot(time, bfcase['L'](time), color = 'green', label = 'bestfit')
ax[1].plot(time, bfcase['G'](time), color = 'green')

ax[0].set_xlim(-0.2, 8.2)
fig.set_size_inches(5, 4)
fig.patch.set_alpha(1.0)
plt.tight_layout()
plt.show()

In [ ]:
gc.collect()

## No activation 

In [ ]:
model_inputs_noact = model_inputs.copy()

In [ ]:
model_inputs_noact['k'] = 0.0

In [ ]:
%%cache model_ensemble_noact.pkl model_ensemble_noact --cachedir ../cache/lg_model/
%%time
model_ensemble_noact = Model_ensemble(model_inputs_noact)

In [ ]:
model_ensemble_noact.describe_initials()

In [ ]:
bfcase = best_fit(model_ensemble_noact, outgrowth_pop, cell_number_data)

time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
model_ensemble_noact.describe(time, fig, ax)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'],
               label = 'exp (fab)', fmt = 's', color = 'black')

# plot bestfits
ax[0].plot(time, bfcase['L'](time), color = 'green', label = 'bestfit')
ax[1].plot(time, bfcase['G'](time), color = 'green')

ax[0].set_xlim(-0.2, 8.2)
fig.set_size_inches(5, 4)
fig.patch.set_alpha(1.0)
plt.tight_layout()
plt.show()

In [ ]:
gc.collect()

## No influx

In [ ]:
model_inputs_noflux = model_inputs.copy()

In [ ]:
model_inputs_noflux['v'] = 0.0

In [ ]:
%%cache model_ensemble_noflux.pkl model_ensemble_noflux --cachedir ../cache/lg_model/
%%time
model_ensemble_noflux = Model_ensemble(model_inputs_noflux)

In [ ]:
model_ensemble_noflux.describe_initials()

In [ ]:
bfcase = best_fit(model_ensemble_noflux, outgrowth_pop, cell_number_data)


time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
model_ensemble_noflux.describe(time, fig, ax)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'],
               label = 'exp (fab)', fmt = 's', color = 'black')

# plot bestfits
ax[0].plot(time, bfcase['L'](time), color = 'green', label = 'bestfit')
ax[1].plot(time, bfcase['G'](time), color = 'green')

ax[0].set_xlim(-0.2, 8.2)
fig.set_size_inches(5, 4)
fig.patch.set_alpha(1.0)
plt.tight_layout()
plt.show()

In [ ]:
gc.collect()

## No flux, no act

In [ ]:
model_inputs_noflux_noact = model_inputs.copy()

In [ ]:
model_inputs_noflux_noact['v'] = 0.0
model_inputs_noflux_noact['k'] = 0.0

In [ ]:
%%cache model_ensemble_noflux_noact.pkl model_ensemble_noflux_noact --cachedir ../cache/lg_model/
%%time
model_ensemble_noflux_noact = Model_ensemble(model_inputs_noflux_noact)

In [ ]:
model_ensemble_noflux_noact.describe_initials()

In [ ]:
bfcase = best_fit(model_ensemble_noflux_noact, outgrowth_pop, cell_number_data)


time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
model_ensemble_noflux_noact.describe(time, fig, ax)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'],
               label = 'exp (fab)', fmt = 's', color = 'black')

# plot bestfits
ax[0].plot(time, bfcase['L'](time), color = 'green', label = 'bestfit')
ax[1].plot(time, bfcase['G'](time), color = 'green')

ax[0].set_xlim(-0.2, 8.2)
fig.set_size_inches(5, 4)
fig.patch.set_alpha(1.0)
plt.tight_layout()
plt.show()

In [ ]:
gc.collect()

## Fit the models

In [ ]:
Gdata = cell_number_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem'])
Gdata.columns = Gdata.columns.droplevel()
Gdata = Gdata.reset_index()

In [ ]:
def chi2(G0, TC1, TC2, switchtime, k, v):
    Tct = lambda t: sp.interp(t, [switchtime, switchtime], [TC1, TC2])
    model_input = {'L0': L0,
                  'G0': G0,
                  'Tct': Tct,
                  'k': k,
                  'v': v}
    L, G = lg_model(model_input, outgrowth_pop['time'])
    L = L[1:]
    G = G[sp.array(outgrowth_pop['time'].apply(lambda x: x in Gdata['time']))]

    outgrowth_fit = outgrowth_pop.query('mean > 0')
    Gdata_fit = Gdata.query('time < 8')
    chi2L = ((L - sp.array(outgrowth_fit['mean']))**2 / sp.array(outgrowth_fit['sem'])**2).sum()
    chi2G = ((G - sp.array(Gdata_fit['mean']))**2 / sp.array(Gdata_fit['sem'])**2).sum()
    return chi2L + chi2G
    

### Find parameter ranges

In [ ]:
fitarg = {}

In [ ]:
def set_fitarg(paramer_name, series):
    description = series.describe(percentiles = [0.0015, 0.9985])
    fitarg[paramer_name] = description['mean']
    fitarg['error_{}'.format(paramer_name)] = fitarg[paramer_name] / 100.0
    fitarg['limit_{}'.format(paramer_name)] = (description['0.1%'], description['99.9%'])
    

#### G0

In [ ]:
set_fitarg('G0', model_ensemble.model_outputs['G0'])


#### TC

In [ ]:
set_fitarg('TC1', model_ensemble.model_outputs['Tct'].apply(lambda f: f(0)))
set_fitarg('TC2', model_ensemble.model_outputs['Tct'].apply(lambda f: f(8)))
fitarg['switchtime'] = 3.5
fitarg['error_switchtime'] = 0.01
fitarg['limit_switchtime'] = [3.0,4.0]



#### k

In [ ]:
set_fitarg('k', model_ensemble.model_outputs['k'])

#### v

In [ ]:
set_fitarg('v', model_ensemble.model_outputs['v'])

### Do the minimization

### Full model

In [ ]:
M = iminuit.Minuit(chi2, errordef = 0.5,
                   fix_switchtime = False, fix_v = False, fix_k = False, fix_G0 = False, fix_TC1 = False, fix_TC2 = False,
                   **fitarg)
M.migrad(ncall = 1000);

### Basal division

In [ ]:
fitarg_basal_div = fitarg.copy()
fitarg_basal_div['TC2'] = 1e99

fitarg_basal_div['switchtime'] = 9.0
fitarg_basal_div['error_switchtime'] = 0.01
fitarg_basal_div['limit_switchtime'] = [8.5,9.5]

M_basal_div = iminuit.Minuit(chi2, errordef = 0.5,
                   fix_switchtime = True, fix_v = False, fix_k = False, fix_G0 = False, fix_TC1 = False, fix_TC2 = True,
                   **fitarg_basal_div)
M_basal_div.migrad(ncall = 1000);

### No division

In [ ]:
fitarg_nodiv = fitarg.copy()
fitarg_nodiv['TC1'] = 1e99
fitarg_nodiv['TC2'] = 1e99
M_nodiv = iminuit.Minuit(chi2, errordef = 0.5,
                   fix_switchtime = True, fix_v = False, fix_k = False, fix_G0 = False, fix_TC1 = True, fix_TC2 = True,
                   **fitarg_nodiv)
M_nodiv.migrad(ncall = 1000);

### No activation

In [ ]:
fitarg_noact = fitarg.copy()
fitarg_noact['k'] = 0.0
M_noact = iminuit.Minuit(chi2, errordef = 0.5,
                   fix_switchtime = False, fix_v = False, fix_k = True, fix_G0 = False, fix_TC1 = False, fix_TC2 = False,
                   **fitarg_noact)
M_noact.migrad(ncall = 1000);

### No influx

In [ ]:
fitarg_noflux = fitarg.copy()
fitarg_noflux['v'] = 0.0
M_noflux = iminuit.Minuit(chi2, errordef = 0.5,
                   fix_switchtime = False, fix_v = True, fix_k = False, fix_G0 = False, fix_TC1 = False, fix_TC2 = False,
                   **fitarg_noflux)
M_noflux.migrad(ncall = 1000);

### No influx, no activation

In [ ]:
fitarg_noflux_noact = fitarg.copy()
fitarg_noflux_noact['v'] = 0.0
fitarg_noflux_noact['k'] = 0.0
M_noflux_noact = iminuit.Minuit(chi2, errordef = 0.5,
                   fix_switchtime = False, fix_v = True, fix_k = True, fix_G0 = False, fix_TC1 = False, fix_TC2 = False,
                   **fitarg_noflux_noact)
M_noflux_noact.migrad(ncall = 1000);

### Plot Fit results

In [ ]:
fitlabels = {M: 'full',
             M_basal_div: 'basal_div',
          M_nodiv: 'nodiv',
         M_noact: 'noact',
         M_noflux: 'noflux',
         M_noflux_noact: 'noflux noact'}

In [ ]:
time = sp.linspace(0, 8, 100)
fig, ax = plt.subplots(ncols = 2, sharex = True)
fig.patch.set_alpha(1.0)

cell_number_plot_data = data.data['cell_number_data']
cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
# ax[0].errorbar(outgrowth['time'], outgrowth['L'], outgrowth['sem'], fmt = 's', color = 'black')
ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
ax[0].set_xlim(0, 9)

# for observer, data in mean_outgrowth.groupby('observer'):
#     ax[0].errorbar(data['time'], data['Length', 'mean'], data['Length', 'sem'], label = observer, fmt = 's')

ax[0].errorbar(outgrowth_pop['time'], outgrowth_pop['mean'], outgrowth_pop['sem'],
               label = 'exp (fab)', fmt = 's', color = 'black')

# best fit
for minuit in [M, M_basal_div, M_nodiv, M_noact, M_noflux, M_noflux_noact]:
    migrad_input = minuit.values
    migrad_input['Tct'] = Tct = lambda t: sp.interp(t,
                                                    [migrad_input['switchtime'], migrad_input['switchtime']],
                                                    [migrad_input['TC1'], migrad_input['TC2']])
    migrad_input['L0'] = L0
    L, G = lg_model(minuit.values, time)
    ax[0].plot(time, L, label = fitlabels[minuit])
    ax[1].plot(time, G)

ax[0].legend(loc = 0)
plt.show()

## Plots

In [ ]:
conditions = ['full', 'basal_div', 'nodiv', 'noact', 'noflux', 'noflux_noact']
ensembles = {'full': model_ensemble,
             'basal_div': model_ensemble_basal_div,
            'nodiv': model_ensemble_nodiv,
            'noact': model_ensemble_noact,
            'noflux': model_ensemble_noflux,
            'noflux_noact': model_ensemble_noflux_noact}

In [ ]:
minuits = {'full': M,
           'basal_div': M_basal_div,
            'nodiv': M_nodiv,
            'noact': M_noact,
            'noflux': M_noflux,
            'noflux_noact': M_noflux_noact}

also save the data needed for plotting

In [ ]:
time = sp.linspace(0, 8, 100)

In [ ]:
pd.Series(time).to_hdf('./lg_model_plot_data.hdf', 'time')
gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
gexpdata.to_hdf('./lg_model_plot_data.hdf', 'gexpdata')
outgrowth_pop.to_hdf('./lg_model_plot_data.hdf', 'outgrowth_pop')
pd.Series(conditions).to_hdf('./lg_model_plot_data.hdf', 'conditions')
for condition in conditions:
    ensemble = ensembles[condition]
    minuit = minuits[condition]
  
    fig, ax = plt.subplots(2, sharex = True, figsize = (3, 6))
    

    cell_number_plot_data = data.data['cell_number_data']
    cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
    
    
    
    # experimental data
    ax[0].errorbar(sp.array(outgrowth_pop['time']), sp.array(outgrowth_pop['mean']),
                   sp.array(outgrowth_pop['sem']),
                   label = 'exp', fmt = 's', color = 'black')
    
    ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
    
    # best fit
    migrad_input = minuit.values
    migrad_input['Tct'] = Tct = lambda t: sp.interp(t,
                                                    [migrad_input['switchtime'], migrad_input['switchtime']],
                                                    [migrad_input['TC1'], migrad_input['TC2']])
    migrad_input['L0'] = L0
    L, G = lg_model(minuit.values, time)
    ax[0].plot(time, L, label = 'model', lw = 2, color = 'black')
    ax[1].plot(time, G,                  lw = 2, color = 'black')
    
    pd.Series(L).to_hdf('./lg_model_plot_data.hdf', 'L_{0}'.format(condition))
    pd.Series(G).to_hdf('./lg_model_plot_data.hdf', 'G_{0}'.format(condition))
    
    # confidence interval
    L_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['99.8']
    L_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['0.1']
    ax[0].fill_between(time, L_3sigma_min, L_3sigma_max, alpha = 0.2, color = 'black')
    pd.Series(L_3sigma_max).to_hdf('./lg_model_plot_data.hdf', 'L_3sigma_max_{0}'.format(condition))
    pd.Series(L_3sigma_min).to_hdf('./lg_model_plot_data.hdf', 'L_3sigma_min_{0}'.format(condition))
    
    G_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['99.8']
    G_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['0.1']
    ax[1].fill_between(time, G_3sigma_min, G_3sigma_max, alpha = 0.2, color = 'black')
    pd.Series(G_3sigma_max).to_hdf('./lg_model_plot_data.hdf', 'G_3sigma_max_{0}'.format(condition))
    pd.Series(G_3sigma_min).to_hdf('./lg_model_plot_data.hdf', 'G_3sigma_min_{0}'.format(condition))    
    
    # formatting
    fig.patch.set_alpha(1.0)
    ax[0].legend(loc = 2)
    ax[0].set_xlim(-0.2, 8.2)
    
    ax[0].tick_params(axis='both', which='major', labelsize='x-large')
    ax[1].tick_params(axis='both', which='major', labelsize='x-large')
    
    ax[1].set_xlabel('time (days)', fontsize = 'xx-large')
    ax[0].set_ylabel('outgrowth ($\mu m$)', fontsize = 'xx-large')
    ax[1].set_ylabel('growth fraction', fontsize = 'xx-large')
    
    ax[0].set_title(condition, fontsize='xx-large')
    
    plt.show()

    

Save some more results data:

In [ ]:
for condition in conditions:
    print condition
    ensemble = ensembles[condition]
    
    # confidence interval
    L_2sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['97.5']
    L_2sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['2.5']
    pd.Series(L_2sigma_max).to_hdf('./lg_model_plot_data.hdf', 'L_2sigma_max_{0}'.format(condition))
    pd.Series(L_2sigma_min).to_hdf('./lg_model_plot_data.hdf', 'L_2sigma_min_{0}'.format(condition))

    G_2sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['97.5']
    G_2sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['2.5']
    pd.Series(G_2sigma_max).to_hdf('./lg_model_plot_data.hdf', 'G_2sigma_max_{0}'.format(condition))
    pd.Series(G_2sigma_min).to_hdf('./lg_model_plot_data.hdf', 'G_2sigma_min_{0}'.format(condition))
    
    L_1sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['84.0']
    L_1sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['16.0']
    pd.Series(L_1sigma_max).to_hdf('./lg_model_plot_data.hdf', 'L_1sigma_max_{0}'.format(condition))
    pd.Series(L_1sigma_min).to_hdf('./lg_model_plot_data.hdf', 'L_1sigma_min_{0}'.format(condition))

    G_1sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['84.0']
    G_1sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['16.0']
    pd.Series(G_1sigma_max).to_hdf('./lg_model_plot_data.hdf', 'G_1sigma_max_{0}'.format(condition))
    pd.Series(G_1sigma_min).to_hdf('./lg_model_plot_data.hdf', 'G_1sigma_min_{0}'.format(condition))
    
    bf = best_fit(ensemble, outgrowth_pop, cell_number_data)
    bf = bf[['G', 'L']]
    best_fit_data = pd.DataFrame()
    best_fit_data['L'] = bf['L'](time)
    best_fit_data['G'] = bf['G'](time)
    best_fit_data.to_hdf('./lg_model_plot_data.hdf', 'best_fit_{0}'.format(condition))
    
    

Plot for presentation at SBMC (musste schnell gehen, code ist hässlich)

In [ ]:
fitlabels = {M: 'full',
          M_nodiv: 'nodiv',
         M_noact: 'noact',
         M_noflux: 'noflux'}

In [ ]:
conditions = ['full', 'nodiv', 'noact', 'noflux', 'noflux_noact']
ensembles = {'full': model_ensemble,
            'nodiv': model_ensemble_nodiv,
            'noact': model_ensemble_noact,
            'noflux': model_ensemble_noflux}
minuits = {'full': M,
            'nodiv': M_nodiv,
            'noact': M_noact,
            'noflux': M_noflux}

In [ ]:
time = sp.linspace(0, 8, 100)
# gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
for condition in ['full']:
    ensemble = ensembles[condition]
    minuit = minuits[condition]
  
    fig, ax = plt.subplots(2, sharex = True, figsize = (3, 6))
    

    cell_number_plot_data = data.data['cell_number_data']
    cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
    
    
    
    # experimental data
    ax[0].errorbar(sp.array(outgrowth_pop['time']), sp.array(outgrowth_pop['mean']),
                   sp.array(outgrowth_pop['sem']),
                   label = 'exp', fmt = 's', color = 'black')
    
#     ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
    
    # best fit
    migrad_input = minuit.values
    migrad_input['Tct'] = Tct = lambda t: sp.interp(t,
                                                    [migrad_input['switchtime'], migrad_input['switchtime']],
                                                    [migrad_input['TC1'], migrad_input['TC2']])
    migrad_input['L0'] = L0
    L, G = lg_model(minuit.values, time)
    ax[0].plot(time, L, label = 'model', lw = 2, color = 'black')
    ax[1].plot(time, G,                  lw = 2, color = 'black')
    
    # confidence interval
    L_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['99.8']
    L_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['0.1']
    ax[0].fill_between(time, L_3sigma_min, L_3sigma_max, alpha = 0.2, color = 'black')
    
    G_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['99.8']
    G_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['0.1']
    ax[1].fill_between(time, G_3sigma_min, G_3sigma_max, alpha = 0.2, color = 'black')
    
    
    # formatting
    fig.patch.set_alpha(1.0)
#     ax[0].legend(loc = 2)
    ax[0].set_xlim(-0.2, 8.2)
    ax[0].set_ylim(0, 4000)
    
    ax[0].tick_params(axis='both', which='major', labelsize='x-large')
    ax[1].tick_params(axis='both', which='major', labelsize='x-large')
    
    ax[1].set_xlabel('time (days)', fontsize = 'xx-large')
    ax[0].set_ylabel('outgrowth ($\mu m$)', fontsize = 'xx-large')
    ax[1].set_ylabel('growth fraction', fontsize = 'xx-large')
    
    ax[0].set_title(condition, fontsize='xx-large')
    
    plt.show()

    

In [ ]:
time = sp.linspace(0, 8, 100)
# gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
for condition in ['full']:
    ensemble = ensembles[condition]
    minuit = minuits[condition]
  
    fig, ax = plt.subplots(2, sharex = True, figsize = (3, 6))
    

    cell_number_plot_data = data.data['cell_number_data']
    cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
    
    
    
    # experimental data
#     ax[0].errorbar(sp.array(outgrowth_pop['time']), sp.array(outgrowth_pop['mean']),
#                    sp.array(outgrowth_pop['sem']),
#                    label = 'exp', fmt = 's', color = 'black')
    
#     ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
    
    # best fit
    migrad_input = minuit.values
    migrad_input['Tct'] = Tct = lambda t: sp.interp(t,
                                                    [migrad_input['switchtime'], migrad_input['switchtime']],
                                                    [migrad_input['TC1'], migrad_input['TC2']])
    migrad_input['L0'] = L0
    L, G = lg_model(minuit.values, time)
    ax[0].plot(time, L, label = 'model', lw = 2, color = 'black')
    ax[1].plot(time, G,                  lw = 2, color = 'black')
    
    # confidence interval
    L_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['99.8']
    L_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['0.1']
    ax[0].fill_between(time, L_3sigma_min, L_3sigma_max, alpha = 0.2, color = 'black')
    
    G_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['99.8']
    G_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['0.1']
    ax[1].fill_between(time, G_3sigma_min, G_3sigma_max, alpha = 0.2, color = 'black')
    
    
    # formatting
    fig.patch.set_alpha(1.0)
#     ax[0].legend(loc = 2)
    ax[0].set_xlim(-0.2, 8.2)
    ax[0].set_ylim(0, 4000)
    
    ax[0].tick_params(axis='both', which='major', labelsize='x-large')
    ax[1].tick_params(axis='both', which='major', labelsize='x-large')
    
    ax[1].set_xlabel('time (days)', fontsize = 'xx-large')
    ax[0].set_ylabel('outgrowth ($\mu m$)', fontsize = 'xx-large')
    ax[1].set_ylabel('growth fraction', fontsize = 'xx-large')
    
    ax[0].set_title(condition, fontsize='xx-large')
    
    plt.show()

    

In [ ]:
time = sp.linspace(0, 8, 100)
# gexpdata = cell_number_plot_data[['ID', 'time', 'G']].groupby('ID').mean().groupby('time').agg(['mean', 'sem']).reset_index()
for condition in ['full']:
    ensemble = ensembles[condition]
    minuit = minuits[condition]
  
    fig, ax = plt.subplots(2, sharex = True, figsize = (3, 6))
    

    cell_number_plot_data = data.data['cell_number_data']
    cell_number_plot_data['G'] = cell_number_plot_data['PCNA'] / cell_number_plot_data['SOX2']
    
    
    
    # experimental data
    ax[0].errorbar(sp.array(outgrowth_pop['time']), sp.array(outgrowth_pop['mean']),
                   sp.array(outgrowth_pop['sem']),
                   label = 'exp', fmt = 's--', color = 'black')
    
#     ax[1].errorbar(gexpdata['time'], gexpdata['G', 'mean'], gexpdata['G', 'sem'], fmt='s', color = 'black')
    
#     # best fit
#     migrad_input = minuit.values
#     migrad_input['Tct'] = Tct = lambda t: sp.interp(t,
#                                                     [migrad_input['switchtime'], migrad_input['switchtime']],
#                                                     [migrad_input['TC1'], migrad_input['TC2']])
#     migrad_input['L0'] = L0
#     L, G = lg_model(minuit.values, time)
#     ax[0].plot(time, L, label = 'model', lw = 2, color = 'black')
#     ax[1].plot(time, G,                  lw = 2, color = 'black')
    
#     # confidence interval
#     L_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['99.8']
#     L_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['L'])['0.1']
#     ax[0].fill_between(time, L_3sigma_min, L_3sigma_max, alpha = 0.2, color = 'black')
    
#     G_3sigma_max = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['99.8']
#     G_3sigma_min = ensemble.describe_function_series(time, ensemble.model_outputs['G'])['0.1']
#     ax[1].fill_between(time, G_3sigma_min, G_3sigma_max, alpha = 0.2, color = 'black')
    
    
    # formatting
    fig.patch.set_alpha(1.0)
#     ax[0].legend(loc = 2)
    ax[0].set_xlim(-0.2, 8.2)
    ax[0].set_ylim(0, 4000)
    
    ax[0].tick_params(axis='both', which='major', labelsize='x-large')
    ax[1].tick_params(axis='both', which='major', labelsize='x-large')
    
    ax[1].set_xlabel('time (days)', fontsize = 'xx-large')
    ax[0].set_ylabel('outgrowth ($\mu m$)', fontsize = 'xx-large')
    ax[1].set_ylabel('growth fraction', fontsize = 'xx-large')
    
    ax[0].set_title(condition, fontsize='xx-large')
    
    plt.show()

    